# Carousel mechanism design
*LuSEE-Night*\
*Antenna Carousel*\
*Joe Silber, LBNL*\
*2022*

Abstract: Assessment of carousel (a.k.a. "turntable") mechanism and torques for the LuSEE-Night lunar instrument. The radio antennas rotate on a platform. Key design parameters and options for the rotation mechanism are considered here.

Except where otherwise specified, all units are in meters, kilograms, and seconds.

## Payload
Total mass budget for the carousel system is 10 kg. Some fraction of this mass (in particular the drive motor and transmission) will be static.

For the purposes of this document, we subdivide the rotating payload into the following estimated masses, moments, and transmission ratios:

In [1]:
import math

plate_outer_diameter = 0.400  # m
plate_inner_diameter = 0.075  # m
plate_thickness = 0.010  # m
plate_density = 2700  # kg/m^3, i.e. aluminum
plate_mass = plate_density * plate_thickness * math.pi/4 * (plate_outer_diameter**2 - plate_inner_diameter**2)
plate_inertia = 

payloads = {'motor internal':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },
            
            'motor external':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },
            
            'main shaft bearing':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },            
            
            'main shaft':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },    
            
            'plate':
                {'mass': plate_mass,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },
            
            'antenna (deployed)':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 4,
                 'ratio': 1.0,
                },
           }

SyntaxError: invalid syntax (3107269786.py, line 8)

## Bearings

### Bearing type and size
The preliminary design (as of 2022-07-07) packaging acommodates a pair of Kaydon KA025AR0 bearings:

![Kaydon KA0xxAR0](kaydon-KA0xxAR0-open-A-bearings.png)

This is a thin section, angular contact, open bearing, with bore 2.5\" (63.5 mm), O.D. 3.0\" (76.2 mm), and thickness 0.25\" (6.35 mm). Mass is low, ~0.12 lbs = 55 g. We utilize the common design pattern of mounting in pairs, to provides tilt resistance.

Stainless steel options (may have custom lead-times and will be more expensive) are available. Replace "K" with "S" in the part number.

Kaydon also offers an attractive "Type X" four-point contact bearing. We have previously used such a bearing (at a much larger scale) in the Dark Energy Spectroscopic Instrument (DESI) ADC rotator mechanism. The type X controls 5 degrees of freedom in a single unit, rather than the typical angular contact duplex. This has the advantage of fewer parts and less weight, as well as removing the need to control thrust preload in the mounting between the two separate bearing units. A type X unit of similar package size as KA025AR0 would be KA025XP0:

![Kaydon KA0xxXP0](kaydon-KA0xxXP0-open-X-bearings.png)

Again, replace "K" with "S" for equivalent stainless option.

### Integrated bearing seals

Various seals are available, which we will utilize, to contain lubricant and exclude lunar dust.

As of 2022-07-07, the precise seal material is to be determined (TBD).

Kaydon's standard seal is nitrile rubber, which we note may need further investigation regarding cryogenic survival temperatures. I would feel most comfortable with polyimide, but need to speak with the vendor.

Seals may be external (between housing and shaft, surrounding the bearing) or integrated in the bearing unit itself. For example, Kaydon offers some models with an integrated double-seal (see table below).

![Kaydon JA0xxXP0](kaydon-JA0xxXP0-sealed-X-bearings.png)

However, the Kaydon line-up does not have sealed bearings in angular contact configurations. 

For sealed bearings, replace "J" with "W" in the part number for the stainless option.

### External bearing seals

If the vendor lacks proven experience with integrated bearing seals in our temperature range (100 K - 400 K), then I prefer that we make our own external seals, from a material with which we have confidence. This way we can move directly to the next phases of testing and integration, as well as have flexibility and visibility as to what is happening.

If we machine our own seals I would use [Vespel SP-3](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-3). This is a MoS<sub>2</sub>-filled polyimide with a long history of use in space applications, with low creep, high strength, and low coefficient of thermal expansion (for a polymer), and which should survive the temperature extremes. I would fix the seal to the housing and have thin, flexible flanges which bear against the shaft. These seals would be riding on the shaft on each side of the bearing.

### Turntable bearings

There are some bearings available with more integrated turntable functionality, for example including an external gear profile. An attractive option may be Kaydon T01-00325EAA:

![Kaydon turntables - basic](kaydon-turntable-bearings-basic.png)

![Kaydon turntables - external gears](kaydon-turntable-bearings-extgear.png)

![Kaydon turntables - illustrations](kaydon-turntable-illustrations.png)

These turntables are not off-the-shelf parts. Cost and lead-time would be TBD. Performance is not particularly any better than the other options; the attraction here would be the compact packaging. The gearing would need some exterior protection from lunar dust.

### Bearings conclusion

*(?) Below, for the purposes of preliminary overall system torque calculations, we will presume selection of a single Type-X, nitrile double-sealed, stainless steel Kaydon bearing, i.e. WA025XP0. (?)*

In [ ]:
# Bearing friction
mu_bearing = 0.

## References
- [Kaydon turntable bearings](https://www.kaydonbearings.com/RealiSlim_TT_bearings.htm)
- [Kaydon sealed, slim bearings](https://www.kaydonbearings.com/RealiSlim_sealed_bearings.htm)